In [1]:
from adapt_utils import replace_symbols

from tree_parser import *

import networkx as nx

import matplotlib
import matplotlib.pyplot as plt

In [2]:
text = file_contents()
database = meta_math_database(text,n=3000)

('included', 3651590, 'tokens from', 'set.mm')
proposition: 3000

In [3]:
def verify2(prop, db, verbose=False):
    tree_graph = list()
    # attempt to verify a proof
    statement = prop.statement
    proof = prop.proof
    #print
    #print 'proof:',proof

    # print some extra stuff for the theorem
    #for hyp in prop.hyps:
    #    print 'hypothesis', hyp.label, concatonate_to_string(hyp.statement), hyp.tree.stringify()
    #print prop.label

    proof_steps_stack = [] # this is equivalent to the statement stack, more or less,
    statement_stack = []
    entails_proof_steps = []

    index = 0 # the index into the proof steps.

    while index<len(proof):
        
        for s in statement_stack:
            print(replace_symbols(" ".join(tree_to_string(s,db,prop))))
        print("")
        #if verbose:
         #print("oi")
        #print 'statement stack:', [t.stringify() for t in statement_stack]
        label = proof[index]

        #was this a hypothesis: arity 0
        if label in prop.e:
            index+=1
            statement_stack.append(prop.e[label].tree)
            proof_steps_stack.append(proof_step(prop.e[label].tree, prop, prop.e[label], []))
            entails_proof_steps.append(proof_steps_stack[-1]) # all e-type statements are entails, I think
        elif label in prop.f:
            index+=1
            statement_stack.append(prop.f[label].tree)
            proof_steps_stack.append(proof_step(prop.f[label].tree, prop, prop.f[label], []))

        # this is a axiom or proposition
        else:
            axiom = db.propositions[label] # or proposition
            arity = axiom.arity()
            if arity>0:
                new_statement = axiom.unify(statement_stack[-arity:],prop)
                tree_graph.append((label, new_statement, statement_stack[-arity:]))
                #print(new_statement)
                #if verbose:
                    #print(new_statement)
                statement_stack = statement_stack[0:-arity]

                new_proof_step = proof_step(new_statement, prop, axiom, proof_steps_stack[-arity:])
                proof_steps_stack = proof_steps_stack[0:-arity]

            else: # deal correctly with the arity 0 propositions
                new_statement = axiom.tree
                #if verbose:
                    #print(new_statement)
                new_proof_step = proof_step(axiom.tree, prop, axiom, [])

            statement_stack.append(new_statement)
            proof_steps_stack.append(new_proof_step)
            if axiom.vclass == '|-':
                entails_proof_steps.append(new_proof_step)

            index+=1
            #print 'current stack:',statement_stack

    
    for s in statement_stack:
        print(replace_symbols(" ".join(tree_to_string(s,db,prop))))
    #print([print(tree_to_string_join(s,db,prop)) for s in statement_stack])
            
    assert len(statement_stack)==1
    #print 'statement:', prop.statement
    #print 'final proof result:', concatonate_to_string(statement_stack[0])
    prop.tree = statement_stack[0]

    # This is questionable.  I'm not really sure why it was in here in the first place, or how the
    # other systems deal with it.
    #prop.entails_proof_steps = entails_proof_steps
    prop.entails_proof_steps = [t for t in entails_proof_steps if t.prop.type != 'e']
    prop.num_entails_proof_steps = len(prop.entails_proof_steps)

    assert statement==[prop.statement[0]]+tree_to_string(prop.tree,db,prop) # The proof returns the desired statement
    return tree_graph

In [4]:
p = database.propositions["dfss2"]
q = database.propositions["bitr4i"]

In [5]:
p.tree.eval(database, p)

'( A C_ B <-> A. x ( x e. A -> x e. B ) )'

In [6]:
prop = p
db = database

tree_graph = list()
# attempt to verify a proof
statement = prop.statement
proof = prop.proof
#print
#print 'proof:',proof

# print some extra stuff for the theorem
#for hyp in prop.hyps:
#    print 'hypothesis', hyp.label, concatonate_to_string(hyp.statement), hyp.tree.stringify()
#print prop.label

proof_steps_stack = [] # this is equivalent to the statement stack, more or less,
statement_stack = []
entails_proof_steps = []

index = 0 # the index into the proof steps.

In [7]:
for _ in range(137):
    #print 'statement stack:', [t.stringify() for t in statement_stack]
    label = proof[index]

    #was this a hypothesis: arity 0
    if label in prop.e:
        index+=1
        statement_stack.append(prop.e[label].tree)
        proof_steps_stack.append(proof_step(prop.e[label].tree, prop, prop.e[label], []))
        entails_proof_steps.append(proof_steps_stack[-1]) # all e-type statements are entails, I think
    elif label in prop.f:
        index+=1
        statement_stack.append(prop.f[label].tree)
        proof_steps_stack.append(proof_step(prop.f[label].tree, prop, prop.f[label], []))

    # this is a axiom or proposition
    else:
        axiom = db.propositions[label] # or proposition
        arity = axiom.arity()
        if arity>0:
            new_statement = axiom.unify(statement_stack[-arity:],prop)
            tree_graph.append((label, new_statement, statement_stack[-arity:]))
            #print(new_statement)
            #if verbose:
                #print(new_statement)
            statement_stack = statement_stack[0:-arity]

            new_proof_step = proof_step(new_statement, prop, axiom, proof_steps_stack[-arity:])
            proof_steps_stack = proof_steps_stack[0:-arity]

        else: # deal correctly with the arity 0 propositions
            new_statement = axiom.tree
            #if verbose:
                #print(new_statement)
            new_proof_step = proof_step(axiom.tree, prop, axiom, [])

        statement_stack.append(new_statement)
        proof_steps_stack.append(new_proof_step)
        if axiom.vclass == '|-':
            entails_proof_steps.append(new_proof_step)

        index+=1
        #print 'current stack:',statement_stack

In [8]:
for ps in proof_steps_stack:
    #print(replace_symbols(" ".join(ps.prop.statement)))
    print(replace_symbols(" ".join(tree_to_string(ps.tree,db, prop))))

A ⊆ B
∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )
∀ x ( x ∈ A → x ∈ B )
( A ⊆ B ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
( ∀ x ( x ∈ A → x ∈ B ) ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )


In [13]:
for ps in proof_steps_stack:
    #print(replace_symbols(" ".join(ps.prop.statement)))
    print ps.tree

<Tree: wss(cA(),cB())>
<Tree: wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx())>
<Tree: wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx())>
<Tree: wb(wss(cA(),cB()),wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx()))>
<Tree: wb(wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx()),wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx()))>


In [43]:
p.entails_proof_steps[2].tree.eval(database,p)

'( A = ( A i^i B ) <-> A = { x | ( x e. A /\\ x e. B ) } )'

In [48]:
p.entails_proof_steps[-1].tree

<Tree: wb(wss(cA(),cB()),wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx()))>

In [49]:
p.entails_proof_steps[-1].prop.tree

<Tree: wb(wph(),wch())>

In [52]:
p.entails_proof_steps[-1].prop.entails_proof_steps[0].tree

<Tree: wb(wps(),wch())>

In [39]:
label

'albii'

In [40]:
s_stack = list(statement_stack)
s_stack

[<Tree: wss(cA(),cB())>,
 <Tree: wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx())>,
 <Tree: wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx())>,
 <Tree: wb(wss(cA(),cB()),wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx()))>,
 <Tree: wb(wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx()),wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx()))>]

In [44]:
s_stack[-1].eval(database,p)

'( A. x ( x e. A -> x e. B ) <-> A. x ( x e. A <-> ( x e. A /\\ x e. B ) ) )'

In [68]:
p.entails_proof_steps[-2].tree.eval(database,p)

'( A. x ( x e. A -> x e. B ) <-> A. x ( x e. A <-> ( x e. A /\\ x e. B ) ) )'

In [104]:
rep_dict = {q.hyps[i].label:s_stack[i] for i in range(len(s_stack)) if q.hyps[i].type == "f"}
rep_dict

{'wch': <Tree: wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx())>,
 'wph': <Tree: wss(cA(),cB())>,
 'wps': <Tree: wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx())>}

In [113]:
print replace_symbols(q.entails_proof_steps[0].tree.copy().replace(rep_dict).eval(database,p))

( ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) ↔ ∀ x ( x ∈ A → x ∈ B ) )


In [110]:
print replace_symbols(q.entails_proof_steps[1].tree.copy().replace(rep_dict).eval(database,p))

( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )


In [116]:
print replace_symbols(q.entails_proof_steps[0].prop.entails_proof_steps[0].tree.copy().replace(rep_dict).eval(database,p))

( ( A ⊆ B ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) ) → ( ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) ↔ A ⊆ B ) )


In [120]:
p.entails_proof_steps[-1].tree

<Tree: wb(wss(cA(),cB()),wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx()))>

In [133]:
p.entails_proof_steps[-2].tree.eval(database,p)

'( A. x ( x e. A -> x e. B ) <-> A. x ( x e. A <-> ( x e. A /\\ x e. B ) ) )'

In [129]:
p.entails_proof_steps[-1].prop.tree

<Tree: wb(wph(),wch())>

In [128]:
#p.entails_proof_steps[-1].unconstrained[0].eval(database,p)

'A. x ( x e. A <-> ( x e. A /\\ x e. B ) )'

In [153]:
for ppp in database.propositions.values()[:1000]:
    if len(ppp.tree.leaves) > 2:
        print ppp.label

rgen2a
rgen2w
3mix1i
3pm3.2i
3mix2i
3mix3i
wral


In [161]:
database.propositions["rgen2a"].tree.leaves

[<Tree: wral(wph(),vy(),cA())>, <Tree: vx()>, <Tree: cA()>]

In [162]:
database.propositions["rgen2a"].hyps

[<f_hypothesis wph: wff ph>,
 <f_hypothesis vx: set x>,
 <f_hypothesis vy: set y>,
 <f_hypothesis cA: class A>,
 <e_hypothesis rgen2a.1: |- |-((xe.A/\ye.A)->ph)>]

In [151]:
database.propositions["nex"].hyps

[<f_hypothesis wph: wff ph>,
 <f_hypothesis vx: set x>,
 <e_hypothesis nex.1: |- |--.ph>]

In [149]:
p.tree.leaves

[<Tree: wss(cA(),cB())>,
 <Tree: wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx())>]

In [115]:
q.entails_proof_steps[0].prop.entails_proof_steps[0].tree

<Tree: wi(wb(wph(),wps()),wb(wps(),wph()))>

In [112]:
p..tree.eval(database,p)

KeyError: 'wph'

In [99]:
q.entails_proof_steps[0].prop.tree

<Tree: wb(wps(),wph())>

In [100]:
s_stack

[<Tree: wss(cA(),cB())>,
 <Tree: wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx())>,
 <Tree: wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx())>,
 <Tree: wb(wss(cA(),cB()),wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx()))>,
 <Tree: wb(wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx()),wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx()))>]

In [ ]:
copy().replace

In [91]:
q.hyps

[<f_hypothesis wph: wff ph>,
 <f_hypothesis wps: wff ps>,
 <f_hypothesis wch: wff ch>,
 <e_hypothesis bitr4i.1: |- |-(ph<->ps)>,
 <e_hypothesis bitr4i.2: |- |-(ch<->ps)>]

In [93]:
q.entails_proof_steps[0].prop.hyps

[<f_hypothesis wph: wff ph>,
 <f_hypothesis wps: wff ps>,
 <e_hypothesis bicomi.1: |- |-(ph<->ps)>]

In [69]:
p.entails_proof_steps[-2].prop.hyps

[<f_hypothesis wph: wff ph>,
 <f_hypothesis wps: wff ps>,
 <f_hypothesis vx: set x>,
 <e_hypothesis albii.1: |- |-(ph<->ps)>]

In [71]:
p.entails_proof_steps[-2].tree

<Tree: wb(wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx()),wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx()))>

In [64]:
q.entails_proof_steps[0].prop.hyps

[<f_hypothesis wph: wff ph>,
 <f_hypothesis wps: wff ps>,
 <e_hypothesis bicomi.1: |- |-(ph<->ps)>]

In [72]:
q.entails_proof_steps[0].tree

<Tree: wb(wps(),wch())>

In [67]:
p.f

{'cA': <f_hypothesis cA: class A>,
 'cB': <f_hypothesis cB: class B>,
 'vx': <f_hypothesis vx: set x>}

In [52]:
#p.proof

In [39]:
for t in s_stack:
    print(replace_symbols(" ".join(tree_to_string(t, database, p))))

A ⊆ B
∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )
∀ x ( x ∈ A → x ∈ B )
( A ⊆ B ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
( ∀ x ( x ∈ A → x ∈ B ) ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )


In [13]:
q.unify(s_stack,p)

<Tree: wb(wss(cA(),cB()),wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx()))>

In [43]:
print(replace_symbols(" ".join(tree_to_string(q.unify(s_stack,p), database, p))))

( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )


In [45]:
database.propositions["ax-mp"].tree

<Tree: wps()>

In [46]:
database.propositions["ax-mp"].hyps

[<f_hypothesis wph: wff ph>,
 <f_hypothesis wps: wff ps>,
 <e_hypothesis min: |- |-ph>,
 <e_hypothesis maj: |- |-(ph->ps)>]

In [15]:
q.hyps

[<f_hypothesis wph: wff ph>,
 <f_hypothesis wps: wff ps>,
 <f_hypothesis wch: wff ch>,
 <e_hypothesis bitr4i.1: |- |-(ph<->ps)>,
 <e_hypothesis bitr4i.2: |- |-(ch<->ps)>]

In [35]:
q.statement

['|-', '(', 'ph', '<->', 'ch', ')']

- Proposition tree leaves shows the variables that need to be set for the proposition
- Proposition hyps shows the variables that compose the proposition
- So to generate the next proposition we just take the variables that are in the leaves of the proposition itself

In [34]:
print replace_symbols(" ".join(tree_to_string(s_stack[2], database, p)))

∀ x ( x ∈ A → x ∈ B )


In [37]:
p.tree

<Tree: wb(wss(cA(),cB()),wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx()))>

In [16]:
rep_dict = {q.hyps[i].label:s_stack[i] for i in range(len(s_stack)) if q.hyps[i].type == "f"}

In [17]:
rep_dict

{'wch': <Tree: wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx())>,
 'wph': <Tree: wss(cA(),cB())>,
 'wps': <Tree: wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx())>}

In [26]:
q.tree.leaves

[<Tree: wph()>, <Tree: wch()>]

In [12]:
q.proof

['wph', 'wps', 'wch', 'bitr4i.1', 'wch', 'wps', 'bitr4i.2', 'bicomi', 'bitri']

In [23]:
saffsa = q.e.values()[0]

In [24]:
saffsa.tree

<Tree: wb(wch(),wps())>

In [30]:
database.propositions["ax-mp"].hyps[-1]

<e_hypothesis maj: |- |-(ph->ps)>

In [14]:
q.f

{'wch': <f_hypothesis wch: wff ch>,
 'wph': <f_hypothesis wph: wff ph>,
 'wps': <f_hypothesis wps: wff ps>}

In [18]:
database.propositions["bicomi"].hyps

[<f_hypothesis wph: wff ph>,
 <f_hypothesis wps: wff ps>,
 <e_hypothesis bicomi.1: |- |-(ph<->ps)>]

In [33]:
database.propositions["ax-3"].hyps

[<f_hypothesis wph: wff ph>, <f_hypothesis wps: wff ps>]

In [32]:
database.propositions["ax-mp"].hyps

[<f_hypothesis wph: wff ph>,
 <f_hypothesis wps: wff ps>,
 <e_hypothesis min: |- |-ph>,
 <e_hypothesis maj: |- |-(ph->ps)>]

In [34]:
database.propositions["wi"].hyps

[<f_hypothesis wph: wff ph>, <f_hypothesis wps: wff ps>]

## Objetivo:
⊢ (∀𝑥(𝑥 ∈ 𝐴 ↔ (𝑥 ∈ 𝐴 ∧ 𝑥 ∈ 𝐵)) ↔∀𝑥(𝑥 ∈ 𝐴 → 𝑥 ∈ 𝐵))

In [ ]:
( ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) ↔ ∀ x ( x ∈ A → x ∈ B ) )

In [7]:
print replace_symbols(" ".join(p.statement))

|- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )


In [9]:
print p.tree
print p.tree.eval(database, p)

<Tree: wb(wss(cA(),cB()),wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx()))>
( A C_ B <-> A. x ( x e. A -> x e. B ) )


In [10]:
print p.tree.leaves[0]
print p.tree.leaves[0].eval(database, p)

<Tree: wss(cA(),cB())>
A C_ B


In [11]:
print p.tree.leaves[1]
print p.tree.leaves[1].eval(database, p)

<Tree: wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx())>
A. x ( x e. A -> x e. B )


In [84]:
p.entails_proof_steps[-1]

<tree_parser.proof_step instance at 0x00000000210A4108>

In [19]:
test_tree = q.hyps[-2].tree.copy().replace(rep_dict)

In [21]:
q.hyps[-2].tree

<Tree: wb(wph(),wps())>

In [20]:
test_tree

<Tree: wb(wss(cA(),cB()),wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx()))>

In [74]:
q.hyps[-1].tree.value

'wb'

In [77]:
s_stack[-1]

<Tree: wb(wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx()),wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx()))>

In [81]:
q.entails_proof_steps[0].prop.label

'bicomi'

In [8]:
print str(index) + "/" + str(len(proof)) + " - " + ", ".join(proof[:index])

137/138 - cA, cB, wss, vx, cv, cA, wcel, vx, cv, cA, wcel, vx, cv, cB, wcel, wa, wb, vx, wal, vx, cv, cA, wcel, vx, cv, cB, wcel, wi, vx, wal, cA, cB, wss, cA, cA, cB, cin, wceq, cA, vx, cv, cA, wcel, vx, cv, cB, wcel, wa, vx, cab, wceq, vx, cv, cA, wcel, vx, cv, cA, wcel, vx, cv, cB, wcel, wa, wb, vx, wal, cA, cB, dfss, cA, cB, cin, vx, cv, cA, wcel, vx, cv, cB, wcel, wa, vx, cab, cA, vx, cA, cB, df-in, eqeq2i, vx, cv, cA, wcel, vx, cv, cB, wcel, wa, vx, cA, abeq2, 3bitri, vx, cv, cA, wcel, vx, cv, cB, wcel, wi, vx, cv, cA, wcel, vx, cv, cA, wcel, vx, cv, cB, wcel, wa, wb, vx, vx, cv, cA, wcel, vx, cv, cB, wcel, pm4.71, albii


A ⊆ B
∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )
∀ x ( x ∈ A → x ∈ B )
( A ⊆ B ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
( x ∈ A → x ∈ B )
( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )
x
x
A


In [9]:
statement_stack

[<Tree: wss(cA(),cB())>,
 <Tree: wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx())>,
 <Tree: wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx())>,
 <Tree: wb(wss(cA(),cB()),wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx()))>,
 <Tree: wb(wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx()),wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx()))>]

In [662]:
database.propositions["bicomi"].proof

['wph',
 'wps',
 'wb',
 'wps',
 'wph',
 'wb',
 'bicomi.1',
 'wph',
 'wps',
 'bicom1',
 'ax-mp']

In [673]:
" ".join(tree_to_string(p.entails_proof_steps[-1].tree, database, p))

'( A C_ B <-> A. x ( x e. A -> x e. B ) )'

In [690]:
" ".join(tree_to_string(p.tree, database, p))

'( A C_ B <-> A. x ( x e. A -> x e. B ) )'

In [691]:
" ".join(tree_to_string(p.entails_proof_steps[-2].tree, database, p))

'( A. x ( x e. A -> x e. B ) <-> A. x ( x e. A <-> ( x e. A /\\ x e. B ) ) )'

In [683]:
p.entails_proof_steps[-1].prop.tree.leaves

[<Tree: wph()>, <Tree: wch()>]

In [681]:
p.entails_proof_steps[-1].tree.leaves

[<Tree: wss(cA(),cB())>,
 <Tree: wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx())>]

In [ ]:
se o negocio so olha as hipoteses do tipo F para criar o replace dict, como ele substitui as outras hipoteses?
preciso entender melhor unify e o replace para extender os metodos e manter as arvores originais apra gerar os intermediarios

In [706]:
q.hyps

[<f_hypothesis wph: wff ph>,
 <f_hypothesis wps: wff ps>,
 <f_hypothesis wch: wff ch>,
 <e_hypothesis bitr4i.1: |- |-(ph<->ps)>,
 <e_hypothesis bitr4i.2: |- |-(ch<->ps)>]

In [705]:
q.tree.leaves

[<Tree: wph()>, <Tree: wch()>]

In [700]:
p.tree

<Tree: wb(wss(cA(),cB()),wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx()))>

In [696]:
p.entails_proof_steps[-2].tree.leaves

[<Tree: wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx())>,
 <Tree: wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx())>]

In [697]:
p.entails_proof_steps[-3].tree.leaves

[<Tree: wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))>,
 <Tree: wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())))>]

In [698]:
p.entails_proof_steps[-4].tree.leaves

[<Tree: wss(cA(),cB())>,
 <Tree: wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx())>]

In [682]:
#tree_to_string(database.propositions["bitr4i"].hyps[-1].tree, database, prop)

In [641]:
statement_stack[0:2] + statement_stack[-1:]

[<Tree: wss(cA(),cB())>,
 <Tree: wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx())>,
 <Tree: wb(wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx()),wal(wb(wcel(cv(vx()),cA()),wa(wcel(cv(vx()),cA()),wcel(cv(vx()),cB()))),vx()))>]

In [642]:
database.propositions["bicomi"].unify(statement_stack[0:2] + statement_stack[-1:], prop)

AssertionError: 

In [628]:
for s in statement_stack:
    print(s.value)

wss
wal
wal
wb
wb


In [629]:
for ps in entails_proof_steps:
    print(replace_symbols(" ".join(ps.prop.statement)))

|- ( A ⊆ B ↔ A = ( A ∩ B ) )
|- ( A ∩ B ) = { x | ( x ∈ A ∧ x ∈ B ) }
|- ( C = A ↔ C = B )
|- ( A = { x | ph } ↔ ∀ x ( x ∈ A ↔ ph ) )
|- ( ph ↔ th )
|- ( ( ph → ps ) ↔ ( ph ↔ ( ph ∧ ps ) ) )
|- ( ∀ x ph ↔ ∀ x ps )


In [525]:
", ".join(proof)

'cA, cB, wss, vx, cv, cA, wcel, vx, cv, cA, wcel, vx, cv, cB, wcel, wa, wb, vx, wal, vx, cv, cA, wcel, vx, cv, cB, wcel, wi, vx, wal, cA, cB, wss, cA, cA, cB, cin, wceq, cA, vx, cv, cA, wcel, vx, cv, cB, wcel, wa, vx, cab, wceq, vx, cv, cA, wcel, vx, cv, cA, wcel, vx, cv, cB, wcel, wa, wb, vx, wal, cA, cB, dfss, cA, cB, cin, vx, cv, cA, wcel, vx, cv, cB, wcel, wa, vx, cab, cA, vx, cA, cB, df-in, eqeq2i, vx, cv, cA, wcel, vx, cv, cB, wcel, wa, vx, cA, abeq2, 3bitri, vx, cv, cA, wcel, vx, cv, cB, wcel, wi, vx, cv, cA, wcel, vx, cv, cA, wcel, vx, cv, cB, wcel, wa, wb, vx, vx, cv, cA, wcel, vx, cv, cB, wcel, pm4.71, albii, bitr4i'

In [527]:
", ".join(database.propositions["bitr4i"].proof)

'wph, wps, wch, bitr4i.1, wch, wps, bitr4i.2, bicomi, bitri'

In [ ]:
need to find how to insert or expand the proof steps of a given theorem

In [523]:
database.propositions["bitr4i"].hyps

[<f_hypothesis wph: wff ph>,
 <f_hypothesis wps: wff ps>,
 <f_hypothesis wch: wff ch>,
 <e_hypothesis bitr4i.1: |- |-(ph<->ps)>,
 <e_hypothesis bitr4i.2: |- |-(ch<->ps)>]

In [521]:
database.propositions["bicomi"].hyps

[<f_hypothesis wph: wff ph>,
 <f_hypothesis wps: wff ps>,
 <e_hypothesis bicomi.1: |- |-(ph<->ps)>]

In [586]:
database.propositions["bitri"].hyps

[<f_hypothesis wph: wff ph>,
 <f_hypothesis wps: wff ps>,
 <f_hypothesis wch: wff ch>,
 <e_hypothesis bitri.1: |- |-(ph<->ps)>,
 <e_hypothesis bitri.2: |- |-(ps<->ch)>]

In [585]:
database.propositions["bitr4i"].entails_proof_steps[1].prop.label

'bitri'

In [61]:
#p.proof = original_proof
#original_proof = list(p.proof)
#p.proof.append('bitri')
#p.proof[-2] = 'bicomi'

In [114]:
p.tree

<Tree: wb(wss(cA(),cB()),wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx()))>

In [118]:
for s in p.entails_proof_steps:
    print(replace_symbols(" ".join(s.prop.statement)))
    print(replace_symbols(" ".join(tree_to_string(s.tree, database, p))))
    print("")

#p.entails_proof_steps[0].context.statement

|- ( A ⊆ B ↔ A = ( A ∩ B ) )
( A ⊆ B ↔ A = ( A ∩ B ) )

|- ( A ∩ B ) = { x | ( x ∈ A ∧ x ∈ B ) }
( A ∩ B ) = { x | ( x ∈ A ∧ x ∈ B ) }

|- ( C = A ↔ C = B )
( A = ( A ∩ B ) ↔ A = { x | ( x ∈ A ∧ x ∈ B ) } )

|- ( A = { x | ph } ↔ ∀ x ( x ∈ A ↔ ph ) )
( A = { x | ( x ∈ A ∧ x ∈ B ) } ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )

|- ( ph ↔ th )
( A ⊆ B ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )

|- ( ( ph → ps ) ↔ ( ph ↔ ( ph ∧ ps ) ) )
( ( x ∈ A → x ∈ B ) ↔ ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )

|- ( ∀ x ph ↔ ∀ x ps )
( ∀ x ( x ∈ A → x ∈ B ) ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )

|- ( ph ↔ ch )
( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )



In [132]:
print(replace_symbols(" ".join(tree_to_string(p.entails_proof_steps[-1].unconstrained[0], database, p))))

∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )


In [120]:
for s in p.entails_proof_steps:
    print(s.prop.label)

dfss
df-in
eqeq2i
abeq2
3bitri
pm4.71
albii
bitr4i


In [135]:
p.entails_proof_steps[-1].tree.leaves

[<Tree: wss(cA(),cB())>,
 <Tree: wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx())>]

In [68]:
def tree_to_string3(tree, database, context):
    if tree.value in context.f: return context.f[tree.value].statement
    prop = database.propositions[tree.value]
    assert len(tree.leaves)==len(prop.hyps)
    replacement_dict = {prop.hyps[i].variable:tree_to_string(tree.leaves[i], database, context) for i in range(len(prop.hyps)) if prop.hyps[i].type == "f"} #find the replacement rules for the variables
    #print 'replacement_dict', replacement_dict, 'applied to', prop.statement
    #print(replacement_dict)
    #print(prop.statement[1:])
    str_r = string_replace(prop.statement[1:],replacement_dict)
    #print_statement(str_r)
    return str_r

In [71]:
print(replace_symbols(" ".join(tree_to_string3(p.tree,database,p))))

( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )


In [92]:
database.propositions['wcel'].statement

['wff', 'A', 'e.', 'B']

In [91]:
#It is not storing the theorems in the tree, just the the scheme to generate the proof statements. 
#It is like a assembly language
#The tree is just a parse tree to structure the expression in order for it to be used for other theorems

In [83]:
p.tree.leaves

[<Tree: wss(cA(),cB())>,
 <Tree: wal(wi(wcel(cv(vx()),cA()),wcel(cv(vx()),cB())),vx())>]

In [116]:
tg = verify2(p, database, False)


A

A
B

A ⊆ B

A ⊆ B
x

A ⊆ B
x

A ⊆ B
x
A

A ⊆ B
x ∈ A

A ⊆ B
x ∈ A
x

A ⊆ B
x ∈ A
x

A ⊆ B
x ∈ A
x
A

A ⊆ B
x ∈ A
x ∈ A

A ⊆ B
x ∈ A
x ∈ A
x

A ⊆ B
x ∈ A
x ∈ A
x

A ⊆ B
x ∈ A
x ∈ A
x
B

A ⊆ B
x ∈ A
x ∈ A
x ∈ B

A ⊆ B
x ∈ A
( x ∈ A ∧ x ∈ B )

A ⊆ B
( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )

A ⊆ B
( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )
x

A ⊆ B
∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )

A ⊆ B
∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )
x

A ⊆ B
∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )
x

A ⊆ B
∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )
x
A

A ⊆ B
∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )
x ∈ A

A ⊆ B
∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )
x ∈ A
x

A ⊆ B
∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )
x ∈ A
x

A ⊆ B
∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )
x ∈ A
x
B

A ⊆ B
∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )
x ∈ A
x ∈ B

A ⊆ B
∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )
( x ∈ A → x ∈ B )

A ⊆ B
∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )
( x ∈ A → x ∈ B )
x

A ⊆ B
∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )
∀ x ( x ∈ A → x ∈ B )

A ⊆ B
∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )
∀ x ( x ∈ A → x ∈ B )
A

A ⊆ B
∀ x 

( x ∈ A ∧ x ∈ B )
x
A

A ⊆ B
∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )
∀ x ( x ∈ A → x ∈ B )
A ⊆ B
A = ( A ∩ B )
A = { x | ( x ∈ A ∧ x ∈ B ) }
∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )
( A ⊆ B ↔ A = ( A ∩ B ) )
( A = ( A ∩ B ) ↔ A = { x | ( x ∈ A ∧ x ∈ B ) } )
( A = { x | ( x ∈ A ∧ x ∈ B ) } ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )

A ⊆ B
∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )
∀ x ( x ∈ A → x ∈ B )
( A ⊆ B ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )

A ⊆ B
∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )
∀ x ( x ∈ A → x ∈ B )
( A ⊆ B ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
x

A ⊆ B
∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )
∀ x ( x ∈ A → x ∈ B )
( A ⊆ B ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
x

A ⊆ B
∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )
∀ x ( x ∈ A → x ∈ B )
( A ⊆ B ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
x
A

A ⊆ B
∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )
∀ x ( x ∈ A → x ∈ B )
( A ⊆ B ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
x ∈ A

A ⊆ B
∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) )
∀ x ( x ∈ A → x ∈ B )
( A ⊆ B ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
x ∈ A
x

A ⊆ B
∀ 

In [26]:
q = database.propositions["bitr4i"]

In [46]:
database.propositions["wph"]

KeyError: 'wph'

In [48]:
q.f

{'wch': <f_hypothesis wch: wff ch>,
 'wph': <f_hypothesis wph: wff ph>,
 'wps': <f_hypothesis wps: wff ps>}

In [28]:
q.proof

['wph', 'wps', 'wch', 'bitr4i.1', 'wch', 'wps', 'bitr4i.2', 'bicomi', 'bitri']

In [29]:
verify2(q, database, False)


ph

ph
ps

ph
ps
ch

ph
ps
ch
( ph ↔ ps )

ph
ps
ch
( ph ↔ ps )
ch

ph
ps
ch
( ph ↔ ps )
ch
ps

ph
ps
ch
( ph ↔ ps )
ch
ps
( ch ↔ ps )

ph
ps
ch
( ph ↔ ps )
( ps ↔ ch )

( ph ↔ ch )


[('bicomi',
  <Tree: wb(wps(),wch())>,
  [<Tree: wch()>, <Tree: wps()>, <Tree: wb(wch(),wps())>]),
 ('bitri',
  <Tree: wb(wph(),wch())>,
  [<Tree: wph()>,
   <Tree: wps()>,
   <Tree: wch()>,
   <Tree: wb(wph(),wps())>,
   <Tree: wb(wps(),wch())>])]

In [30]:
q.tree

<Tree: wb(wph(),wch())>

In [38]:
for ps in p.entails_proof_steps:
    print(replace_symbols(" ".join(ps.prop.statement)))

|- ( A ⊆ B ↔ A = ( A ∩ B ) )
|- ( A ∩ B ) = { x | ( x ∈ A ∧ x ∈ B ) }
|- ( C = A ↔ C = B )
|- ( A = { x | ph } ↔ ∀ x ( x ∈ A ↔ ph ) )
|- ( ph ↔ th )
|- ( ( ph → ps ) ↔ ( ph ↔ ( ph ∧ ps ) ) )
|- ( ∀ x ph ↔ ∀ x ps )
|- ( ph ↔ ch )


In [36]:
ps.prop.statement

['|-', '(', 'A', 'C_', 'B', '<->', 'A', '=', '(', 'A', 'i^i', 'B', ')', ')']

In [43]:
tree_to_string2(p.tree, database, p)

(['(',
  'A',
  'C_',
  'B',
  '<->',
  'A.',
  'x',
  '(',
  'x',
  'e.',
  'A',
  '->',
  'x',
  'e.',
  'B',
  ')',
  ')'],
 {'ph': ['A', 'C_', 'B'],
  'ps': ['A.', 'x', '(', 'x', 'e.', 'A', '->', 'x', 'e.', 'B', ')']})